In [ ]:
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re
import operator

nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk_stopwords= set(stopwords.words('english'))

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [ ]:
# !pip list

In [2]:
df= pd.read_json('/content/drive/MyDrive/amazon_data_scrapping/Data files/Potporri_Reviews.json', orient='index')
df.head()

,reviewDate,reviewRating,reviewText,reviewTitle
0,"Reviewed in the United States on November 23, 2019",5.0 out of 5 stars,"Bought 3 of these to fill a vase/jar in my bathroom. Color did go well with my bathroom setting. Scent smell is good enough and keeping the bad odor out of my bathroom. It worked out for me.\n\nused 4.5 packs to fill the jar (11 inches height , 5.5 inches diameter)",It suited and worked out well with my bathroom setting!!!!
1,"Reviewed in the United States on September 7, 2019",4.0 out of 5 stars,"I got the blue ones, 2 pack. They're beautiful and smells great but the scent lasted maybe 2 weeks. I kept them and plan to put essential oils on them and see how that does. I love the look just wish it held the scent longer.",They're pretty but wishes they held the scent longer
2,"Reviewed in the United States on December 17, 2020",1.0 out of 5 stars,"I bought the ""Rose"" scented package. There were a decent amount of dried flowers and other ingredients. The package came with a VERY small bottle of scent that you have to sprinkle onto the dried items in order to get a scent. The quality of the Rose scent is actually quite nice, with no odd chemical odors. That is a great positive for this potpourri. However, that nice and light scent doesn't last very long. I made two small bowls and used about half of the VERY small bottle of scent between the two bowls. The scent lasted a bit over a week, and that was it. Once the scent evaporates there is nearly no scent left. Back a couple of decades ago when potpourri seemed much more popular I could buy big bags locally. And their scent would last months, and the scent was infused into the dried flowers and other dried items with no need to sprinkle a scent oil on. I've had some potpourri that is pre-infused that I could still smell nearly a YEAR later. This product not at all. I expect a potpourri to have a nice scent that lasts at least 2-3 months with a nice intensity of scent. That does not happen with this potpourri, not at all. I will not be purchasing this potpourri again as it is too expensive for a scent that only last about 2 weeks.",Too expensive for very short live scent.
3,"Reviewed in the United States on June 5, 2020",5.0 out of 5 stars,"A product looks appealing with an alluring aroma. A man found a small empty bottle of scented oil in a package, he is not sure why. A man also discovers potpourri means “rotten pot” in French.",Alluring
4,"Reviewed in the United States on February 5, 2021",5.0 out of 5 stars,"I am usually an ""anti-scent"" person as most scents give me a headache. But, my mom really likes having things like this in her room for a ""pop of color"" as she puts it. I didn't realize when I ordered, but was pleased to find that there actually isn't a scent on it at first. It comes with a tiny bottle of oil (I think it's oil - some kind of liquid) that is the scent, so you can put on as little or as much as you want! Yay! Mom can have her pop of color and I can have my scent free home, win-win. Also, it's very pretty! Take a minute or two to pick out your favorite pieces and put them on top so you can enjoy them the most. Good find!",Very nice


In [3]:
df['reviewDate']= df['reviewDate'].apply(lambda date:re.sub('Reviewed in the United States on ',"", date).strip())
df['reviewRating']= df['reviewRating'].apply(lambda rating: re.sub(' out of 5 stars',"", rating).strip()) 
df['reviewRating']= df.reviewRating.astype('float')
df['reviewClass']=df.apply(lambda row: 1 if row.reviewRating>=4.0 else 0, axis=1)
# df['reviewText']= df['reviewTitle'] + '. '+ df['reviewText']
df['reviewText']= df['reviewText'].apply(lambda review: review.lower())
df['reviewText'] = df['reviewText'].apply(lambda review:re.sub("\n", ' ', review))

#shortforms
df['reviewText']= df['reviewText'].apply(lambda review: re.sub('bc|b/c', 'because', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("wasn’t", 'was not', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("i’m", 'i am', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("i'm", 'i am', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("doesn’t", 'does not', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("doesn't", 'does not', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("didn’t", 'did not', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("won't", 'will not', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("won’t", 'will not', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("don’t", 'do not', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("can't", 'can not', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("it's", 'it is', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("it’s", 'it is', review))
# df['reviewText']= df['reviewText'].apply(lambda review: re.sub("(\\d|\\W)+", ' ', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("(\\d)+", ' ', review))

In [4]:
df.shape

(670, 5)

In [ ]:
df['reviewText']

In [6]:
# reviews=df[df['sentiment']==1]['reviewText']
reviews=df[df['reviewClass']==1]['reviewText']
positive_reviews=''
for review in reviews:
  positive_reviews= positive_reviews+" "+review



In [7]:
# reviews=df[df['sentiment']==0]['reviewText']
reviews=df[df['reviewClass']==0]['reviewText']
negative_reviews=''
for review in reviews:
  negative_reviews= negative_reviews+" "+review

In [8]:
positive_reviews

'  it suited and worked out well with my bathroom setting!!!!. bought   of these to fill a vase/jar in my bathroom. color did go well with my bathroom setting. scent smell is good enough and keeping the bad odor out of my bathroom. it worked out for me.  used  .  packs to fill the jar (  inches height ,  .  inches diameter)  they\'re pretty but wishes they held the scent longer. i got the blue ones,   pack. they\'re beautiful and smells great but the scent lasted maybe   weeks. i kept them and plan to put essential oils on them and see how that does. i love the look just wish it held the scent longer.  alluring. a product looks appealing with an alluring aroma. a man found a small empty bottle of scented oil in a package, he is not sure why. a man also discovers potpourri means “rotten pot” in french.  very nice. i am usually an "anti-scent" person as most scents give me a headache. but, my mom really likes having things like this in her room for a "pop of color" as she puts it. i didn

In [9]:
def create_frequency_table(reviews):

  #Use stemmer
  words= word_tokenize(reviews)
  porter_stemmer= PorterStemmer()
  lemmatizer= WordNetLemmatizer()

  frequency_table= {}
  for word in words:
    word= lemmatizer.lemmatize(word)
    if word in nltk_stopwords:
      continue
    if word in frequency_table:
      frequency_table[word]+= 1
    else:
      frequency_table[word]=1
  return frequency_table


In [10]:
def rank_sentences(positive_reviews, frequency_table):
  sentences= sent_tokenize(positive_reviews)
  sentence_scores= {}
  for sentence in sentences:
    word_count_in_sentence= len(word_tokenize(sentence))
    for key in frequency_table:
      if key in sentence.lower():
        if sentence[:20] in sentence_scores:
          sentence_scores[sentence[:20]] += frequency_table[key]
        else:
          sentence_scores[sentence[:20]] = frequency_table[key]

    sentence_scores[sentence[:20]] = sentence_scores[sentence[:20]] // word_count_in_sentence
  return sentence_scores

In [11]:
def generate_summary(sentence_scores, positive_reviews):
  total_sum=0
  summary= ''

  for value in sentence_scores.values():
    total_sum=total_sum+value
  cut_off_score= total_sum// len(sentence_scores)

  cut_off_score= 1.5 * cut_off_score

  sentences= sent_tokenize(positive_reviews)
  for sentence in sentences:
    if sentence[:20] in sentence_scores and sentence_scores[sentence[:20]] >cut_off_score:
      summary+= ' '+ sentence
  
  return summary

In [12]:
frequency_table=create_frequency_table(positive_reviews)
sentence_scores=rank_sentences(positive_reviews, frequency_table)
nltk_summary=generate_summary(sentence_scores,positive_reviews)

In [13]:
nltk_summary

' used  . alluring. very nice. perfect product. the fragrance. almost   stars!. nice. blue ocean potpourri. great color. almost   stars!. nice. blue ocean potpourri. great color. love it. fast delivery!. great buy. good scent. amazing scent. love the scent. pleasant smell. fast delivery!. great buy. good scent. amazing scent. love the scent. good color. enhances the smell nicely. smells and looks beautiful. nothing was smashed. smells wonderful!. good color. enhances the smell nicely. smells and looks beautiful. nothing was smashed. smells wonderful!. great product. great job  nice color. beautiful scent. gud btru!. great product. great job  nice color. beautiful scent. gud btru!. love it!. great refreshing scent. wow!. beautiful mix. very happy. they smell so good. stunning. beautiful mix. very happy. they smell so good. stunning. nice product. nice  love it. smells good. looks pretty. smells good  smell beautifully. wonderful smelling potpourri!. nice product. nice  love it. smells g

In [ ]:
#change to camelcase
#apply spell check
#clean up extra spaces and periods


# remove similar sentences
# run this summary through the algorithm again to further crisp the summary
# check length of reviews and decide on some cutoff. Reviews below the cutoff should not be considered to ensure review quality


Gensim Summary

In [ ]:
from gensim.summarization import summarize
gensim_summary= summarize(positive_reviews, ratio=0.05)
print(gensim_summary)
# summary_list = gensim_summary.split('.')
# for i,v in enumerate(summary_list):
#   summary_list[i]= v.strip() + '.'
# summary_list

In [15]:
gensim_summary

"scent smell is good enough and keeping the bad odor out of my bathroom.\nthey're beautiful and smells great but the scent lasted maybe   weeks.\npotpourri was small & cute for what i needed it for to fill a dish in my bathroom which didn't have bulky big items in it and that is what i wanted, it also came with a very small bottle of fragrance, which didn't last long-but all in all it covered what i needed it for  smells wonderful.\nthe turquoise color looks even better in person and the smell of this potpourri is amazing.\nthe aroma is not strong, but i also use scented candles, so i am not dependent on the potpourri for a pleasant smell.\nthe aroma is not strong, but i also use scented candles, so i am not dependent on the potpourri for a pleasant smell.\nbeautiful scent, my daughter loved the rose scent , only if the scent lasted longer, a tiny bottle was included , many if that six drops i got out of that tiny bottle for two nice size bags of potpourri not enough scent oil.\nbeauti

Bert Summary

In [ ]:
!pip install bert-extractive-summarizer
!pip install sentencepiece

In [22]:
from summarizer import Summarizer
model = Summarizer()
model(positive_reviews, ratio=0.05)

'it suited and worked out well with my bathroom setting!!!!. price point was very fair given that you received a bottle of the liquid fragrance. i do have to find a nicer bowl to display it in now! bigger bottles of scent oil are differently needed , since the scent only last a many an hour.. beautiful scent, my daughter loved the rose scent , only if the scent lasted longer, a tiny bottle was included , many if that six drops i got out of that tiny bottle for two nice size bags of potpourri not enough scent oil. i also read it is toxic to cats and although i have three they’re very well fed and have plenty of toys they weren’t curious to eat or play w/chips! lovely scent but does not last long.. beautiful pot pourri with bright colors and smells very nice. it was well packaged and it is a pretty accent to the bowl i have on my table. beautiful potpourri, long lasting scent  beautiful scent and came with oil so i can refresh it.. i love the scent of orange and the fact that it came wit

Appendix

sentiment with nltk but reviewText as input instead of reviewTitle

In [ ]:
df['positive']= df['reviewText'].apply(lambda review: sia_object.polarity_scores(review)['pos'])
df['negative']= df['reviewText'].apply(lambda review: sia_object.polarity_scores(review)['neg'])
df['neutral']= df['reviewText'].apply(lambda review: sia_object.polarity_scores(review)['neu'])
df['compound']= df['reviewText'].apply(lambda review: sia_object.polarity_scores(review)['compound'])
df['sentiment']= df['compound'].apply(lambda score: get_sentiment(score))
df.head()

In [ ]:
# 154 positive reviews
df[df['sentiment']==1].shape

#6 negative reviews
df[df['sentiment']==0].shape

In [ ]:
df[(df['reviewRating']==5) | (df['reviewRating']==4)].shape

In [ ]:
df[(df['reviewRating']==3) | (df['reviewRating']==2) | (df['reviewRating']==1)].shape

In [ ]:
would_recommend=sum(df[df['sentiment']==1]['compound'])
would_recommend

In [ ]:
would_not_recommend=sum(df[df['sentiment']==0]['compound'])
would_not_recommend

In [ ]:
ratio= would_recommend/ would_not_recommend
ratio

In [ ]:
sum_positive= sum( df['positive'])
print( sum_positive)

sum_negative= sum( df['negative'])
print( sum_negative)

sum_neutral= sum( df['neutral'])
print( sum_neutral)

print()


In [ ]:
sia_object.polarity_scores('like a piece of art in my kitchen. i just purchased new knives and was so tired of having a big block of wood on the counter filled with many knives that i never used. after years of using the same knives, i purchased new ones of the the sizes that i commonly use. my new knives look beautiful in this holder. everyone comments on it when they see it.')

In [ ]:
df[['sentiment', 'reviewRating', 'reviewText', 'compound']].iloc[[17]]